In [1]:
!pyenv version

3.6.8/envs/combo3 (set by /Users/mkumada/Documents/class/2020/work/combo3/.python-version)


In [2]:
import sys
import pprint
sys.path.append('/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/')
pprint.pprint(sys.path)

['/Users/mkumada/Documents/class/2020/work/combo3/mywork',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles/lib/python',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/combo3/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/combo3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/']


In [3]:
# -*- coding: utf-8 -*-
import sys

import numpy as np
# import cPickle as pickle
import _pickle as cPickle
import scipy
import combo
import os
import urllib
import time
from combo.variable import variable

In [4]:
def load_data():
    # current_dir = os.path.dirname(os.path.abspath(sys.argv[0]))
    current_dir = './'
    arr = np.genfromtxt(os.path.join(current_dir, 'data.csv'), skip_header=1, delimiter=',')
    # 対象列に値が埋められているサンプルだけを学習データとして抽出します。

    arr_train = arr[~np.isnan(arr[:, 0]), :]
    arr_test = arr[np.isnan(arr[:, 0]), :]
    X_train = arr_train[:, 1:]
    # 既に検討した候補
    t_train = arr_train[:, 0]


    # 対象列が空であるサンプルをテストデータとして抽出します。
    X_test = arr_test[:, 1:]
    # 次に検討すべき候補
    # テストデータのインデックスのリストを取得します。
    # 元となった csv ファイルの行番号に合わせるため、+2 しておきます。
    test_idx_list = np.where(np.isnan(arr[:, 0]))[0].tolist()
    test_idx_list = [i + 2 for i in test_idx_list]

    X_all=arr[:, 1:]

    return X_train, t_train, X_test, test_idx_list, X_all


In [5]:
X_train, t_train, X_test, test_idx_list, X_all = load_data()
print(X_train)
print(X_test)

[[0.56       0.13856406]
 [0.7        0.17320508]
 [0.46       0.31176915]
 [0.76       0.34641016]
 [0.5        0.45033321]]
[[0.         0.        ]
 [0.04       0.        ]
 [0.08       0.        ]
 [0.12       0.        ]
 [0.16       0.        ]
 [0.2        0.        ]
 [0.24       0.        ]
 [0.28       0.        ]
 [0.32       0.        ]
 [0.36       0.        ]
 [0.4        0.        ]
 [0.44       0.        ]
 [0.48       0.        ]
 [0.52       0.        ]
 [0.56       0.        ]
 [0.6        0.        ]
 [0.64       0.        ]
 [0.68       0.        ]
 [0.72       0.        ]
 [0.76       0.        ]
 [0.8        0.        ]
 [0.84       0.        ]
 [0.88       0.        ]
 [0.92       0.        ]
 [0.96       0.        ]
 [1.         0.        ]
 [0.02       0.03464102]
 [0.06       0.03464102]
 [0.1        0.03464102]
 [0.14       0.03464102]
 [0.18       0.03464102]
 [0.22       0.03464102]
 [0.26       0.03464102]
 [0.3        0.03464102]
 [0.34       0.03464102]

In [6]:
# Design of policy
if __name__=="__main__":
    # Load the data.
    # X is the N x d dimensional matrix. Each row of X denotes the d-dimensional feature vector of search candidate.
    # t is the N-dimensional vector that represents the corresponding negative energy of search candidates.
    # ( It is of course unknown in practice. )

    X_train, t_train, X_test, test_idx_list, X_all = load_data()

    # Normalize the mean and standard deviation along the each column of X to 0 and 1, respectively
    #X = combo.misc.centering(X)

    # feature vector の各要素を正規化します。
    ave = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train = (X_train - ave) / (std + 1e-8)
    X_test = (X_test - ave) / (std + 1e-8)

    train_data = variable(X=X_train, t=t_train)
    test_data = variable(X=X_test)

    print('training:', train_data)

    # Declare the class for calling the simulator.
    # In this tutorial, we simply refer to the value of t.
    # If you want to apply combo to other problems, you have to customize this class.
    # Declaring the policy by
    policy = combo.search.discrete.policy(test_X=test_data)
    # test_X is the set of candidates which is represented by numpy.array.
    # Each row vector represents the feature vector of the corresponding candidate

    # set the seed parameter
    policy.set_seed(0)

    # If you want to perform the initial random search before starting the Bayesian optimization,
    # the random sampling is performed by

    # 最初のランダムサーチは行わない想定なのでコメントアウトします。
    #res = policy.random_search(max_num_probes=20, simulator=simulator())
    # Input:
    # max_num_probes: number of random search
    # simulator = simulator
    # output: combo.search.discreate.results (class)

    # single query Bayesian search
    # The single query version of COMBO is performed by

    # テストデータ全体の中から次に評価すべき一点だけを提示するメソッドに差し替えて使用します。
    #res = policy.bayes_search(max_num_probes=1, simulator=simulator(), score='TS',
    #                          interval=20, num_rand_basis=5000)

    action = policy.bayes_search_single(training=train_data, score='TS', num_rand_basis=5000)

    # Input
    # max_num_probes: number of searching by Bayesian optimization
    # simulator: the class of simulator which is defined above
    # score: the type of aquision funciton. TS, EI and PI are available
    # interval: the timing for learning the hyper parameter.
    # In this case, the hyper parameter is learned at each 20 steps
    # If you set the negative value to interval, the hyper parameter learning is not performed
    # If you set zero to interval, the hyper parameter learning is performed only at the first step
    # num_rand_basis: the number of basis function. If you choose 0,  ordinary Gaussian process runs

    # save the results
    #res.save('test.npz')
    # テストデータの行番号リストの中からaction番目のものを提示する


    print("Next Point: " + str(X_all[test_idx_list[action]-2]) + "   Row Number: " + str(test_idx_list[action]))

# sys.exit()
# time.sleep(1e2)

training: <combo.variable.variable object at 0x11a57bf98>
Fitting starts
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 29.203724958978132
50-th epoch, marginal likelihood 29.17980475930468
100-th epoch, marginal likelihood 29.1685270712508
150-th epoch, marginal likelihood 29.163160761847283
200-th epoch, marginal likelihood 29.160817878344876
250-th epoch, marginal likelihood 29.159790550670447
300-th epoch, marginal likelihood 29.159264891444334
350-th epoch, marginal likelihood 29.158913736700896
400-th epoch, marginal likelihood 29.158623817771534
450-th epoch, marginal likelihood 29.15836014197323
500-th epoch, marginal likelihood 29.158112069236395
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.]
 scale  =  1.0
 scale2 =  1.0
 

Fitting ends
Calculating score for test data
Next Point: [0.74       0.45033321]   Row Number: 274
